### Imports
---

In [1]:
from keras import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn import preprocessing

Fetching csv file

In [2]:
data_df = pd.read_csv("./admission_data.csv")
data_df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332,108,5,4.5,4.0,9.02,1,0.87
496,497,337,117,5,5.0,5.0,9.87,1,0.96
497,498,330,120,5,4.5,5.0,9.56,1,0.93
498,499,312,103,4,4.0,5.0,8.43,0,0.73


Encoding functions

In [3]:
import collections

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(
        target_type, collections.Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)


Removing spaces in columns for compatibility and removing serial no. column

In [4]:
data_df.columns = data_df.columns.str.replace(' ', '')
data_df = data_df.drop('SerialNo.', axis=1)

Creating test data

In [5]:
testData = [[300, 110, 3, 4.0, 4.0, 8.40, 1, 0.80],
            [327, 120, 4, 2.0, 3.5, 9.50, 0, 0.78]]
testData = pd.DataFrame(testData, columns=data_df.columns)
testData

,GREScore,TOEFLScore,UniversityRating,SOP,LOR,CGPA,Research,ChanceofAdmit
0,300,110,3,4.0,4.0,8.4,1,0.80
1,327,120,4,2.0,3.5,9.5,0,0.78


Replaces the chance of admit column with binary values for admission possiblity over 80 percent

In [6]:
data_df['ChanceofAdmit'] = pd.DataFrame(
    np.where(data_df['ChanceofAdmit'] > 0.8, 1, 0))

testData['ChanceofAdmit'] = pd.DataFrame(
    np.where(testData['ChanceofAdmit'] > 0.8, 1, 0))

Splitting the data into the X and Y format for tensorflow

In [17]:
X, Y = to_xy(data_df, 'ChanceofAdmit')
testX, testY = to_xy(testData, 'ChanceofAdmit')

Fitting the data to a model

In [18]:
model = Sequential()
model.add(Dense(12, input_dim=X.shape[1], activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, Y, verbose=2, epochs=500)

Epoch 1/500
16/16 - 0s - loss: 1.7599
Epoch 2/500
16/16 - 0s - loss: 0.6131
Epoch 3/500
16/16 - 0s - loss: 0.5053
Epoch 4/500
16/16 - 0s - loss: 0.4985
Epoch 5/500
16/16 - 0s - loss: 0.4903
Epoch 6/500
16/16 - 0s - loss: 0.4848
Epoch 7/500
16/16 - 0s - loss: 0.4854
Epoch 8/500
16/16 - 0s - loss: 0.4830
Epoch 9/500
16/16 - 0s - loss: 0.4711
Epoch 10/500
16/16 - 0s - loss: 0.4695
Epoch 11/500
16/16 - 0s - loss: 0.4998
Epoch 12/500
16/16 - 0s - loss: 0.4640
Epoch 13/500
16/16 - 0s - loss: 0.4564
Epoch 14/500
16/16 - 0s - loss: 0.4496
Epoch 15/500
16/16 - 0s - loss: 0.4553
Epoch 16/500
16/16 - 0s - loss: 0.4439
Epoch 17/500
16/16 - 0s - loss: 0.4370
Epoch 18/500
16/16 - 0s - loss: 0.4421
Epoch 19/500
16/16 - 0s - loss: 0.4396
Epoch 20/500
16/16 - 0s - loss: 0.4301
Epoch 21/500
16/16 - 0s - loss: 0.4197
Epoch 22/500
16/16 - 0s - loss: 0.4493
Epoch 23/500
16/16 - 0s - loss: 0.4709
Epoch 24/500
16/16 - 0s - loss: 0.4337
Epoch 25/500
16/16 - 0s - loss: 0.4519
Epoch 26/500
16/16 - 0s - loss: 0.

In [19]:
pred = model.predict(testX)
pred = np.argmax(pred, axis=0)
true = np.argmax(testY, axis=1)

Import for accuracy and classification

In [10]:
from sklearn.metrics import accuracy_score, classification_report

True is reshaped so that it can work with accuracy score and classification

In [20]:
true = true.reshape(-1, 1)

In [21]:
print('Accuracy on test data is %.2f' % (accuracy_score(true, pred)))
print('-------------------------------')
print(classification_report(true, pred))


Accuracy on test data is 0.50
-------------------------------
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



C:\Users\qtrex\anaconda3\envs\annn\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\qtrex\anaconda3\envs\annn\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\qtrex\anaconda3\envs\annn\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
